In [ ]:
include("../code/sp.jl")
include("../code/matrix_generator.jl")
include("../code/slim_graphs.jl")
include("../code/bp_full.jl")
include("../code/bp.jl")
include("../../telegram/notifications.jl")
using ProgressMeter, StaticArrays, OffsetArrays, Statistics, Base.Threads

In [2]:
callback(t, ε, sp) = (println("$t $ε"); false)

function decimate!(sp, indep; decsteps=1, callback=callback)
    H = sp.H
    freevars = copy(indep)
    iteration!(sp, maxiter=700, tol=1e-5, damp=0.5)#, callback=callback)
    while !isempty(freevars)
        iteration!(sp, maxiter=150, tol=1e-3, damp=0.5, callback=callback)
        mag = map(p->p'eachindex(p), sp.survey)
        sort!(freevars, by=i->abs(mag[i]))
        freevars, tofix = freevars[1:end-decsteps], freevars[max(1,end-decsteps+1):end]
        sp.efield[tofix] .= sp.J.*sign.(mag[tofix])
#         println("Free: ", length(freevars))
    end
end

decimate! (generic function with 3 methods)

In [3]:
function sp_dec(R, n; y=0.5, Tmax=1, J = 20, callback=(x...)->false)
    m = round(Int, n*(1-R))
    r = 1-3R
    Λ = OffsetVector([0,0,1-r,r], 0:3)
    K = OffsetVector([0,0,0,1], 0:3)
    nedges = 3m
    ds = fill(NaN, Tmax)
    ps = fill(typemax(Int), Tmax)
    for t in 1:Tmax
        print("Trial $t of $Tmax: ")
        H = permutedims(ldpc_matrix(n,m,nedges,Λ,K))
        s = rand((-1,1), n)
        B, indep = findbasis_slow(Array(H))
        init=OffsetArray(MVector{2J+1}(rand(2J+1)), -J:J)
        sp = survey_propagation(H; field=copy(s), init, y=y);
        decimate!(sp, indep, callback=callback)
        nunsat, ovl, dist = performance(sp, s)
        ps[t] = nunsat
        print("nunsat=$nunsat. dist=", round(dist,digits=3))
        x = sign.(argmax.(sp.survey)) .== -1
        fix_indep!(x, B, indep)   
#         @assert parity(H,x)==0
        σ = (-1).^x
        ds[t] = distortion(σ,s)
        println(". After fix indep, dist=", round(ds[t], digits=3))
        flush(stdout)
    end
    t = argmin(ps)
    ds[t]
end

sp_dec (generic function with 1 method)

In [ ]:
n = 600
R = 0.25
J = 20
y = 0.5
Tmax = 2
dist = sp_dec(R, n, J=J, y=y, Tmax=Tmax, callback=x->false)

Trial 1 of 2: 

In [ ]:
plot_rdb(;f3=true)
scatter!([R], [dist], label="SP+dec, n=$n")

In [ ]:
function y_optimal(R::Real)
    yopt_data = readdlm("yopt_data.txt")
    Rs_rsb = yopt_data[1,:]
    y_rsb = yopt_data[2,:]
    @assert issorted(Rs_rsb, rev=true)
    i = findfirst(isequal(R), Rs_rsb)
    if i === nothing
        for j in eachindex(Rs_rsb)
            if Rs_rsb[j] < R 
                j == 1 && return y_rsb[j]
                xa, xb = Rs_rsb[j-1], Rs_rsb[j]
                ya, yb = y_rsb[j-1], y_rsb[j]
                m = (yb-ya)/(xb-xa)
                return ya + m*(R - xa)
#                 return (y_rsb[j]+y_rsb[j-1])/2
            end
        end
    else
        return y_rsb[i]
    end
end

In [ ]:
n = 600
J = 20
Tmax = 20
f2s = [.4, .45, .5, .55, .6, .65, .7, .75, .8, .85, .9, .95]
# f3s = 0.2:0.05:0.6
f3s = 1 .- f2s
navg = 20
D_sid = [fill(NaN, navg) for _ in f3s]
R_sid = (1 .- f3s)/3
ys = y_optimal.(R_sid);

In [ ]:
@telegram "SP" for i in eachindex(f3s)
    println("Rate $i of ", length(f3s))
    flush(stdout)
    @threads for j in 1:navg
        D_sid[i][j] = sp_dec(R_sid[i], n, J=J, y=ys[i], Tmax=Tmax)
    end
end

In [ ]:
plot_rdb(; f3=true)
ddd = [[minimum(d) for d in dd] for dd in D_sid]
scatter!(R_sid, mean.(ddd), label="SP+decimation, optimal y", c=:purple, ms=3,
    yerr=std.(ddd)/sqrt(navg), size=(400,300), grid=false)

In [9]:
using DelimitedFiles
open("sp_dec_zeroleaves.txt", "w") do f   
   writedlm(f, [R_sid mean.(ddd) std.(ddd)/sqrt(navg)])
end

In [ ]:
mean.(ddd)

In [ ]:
R_sid

In [ ]:
savefig("sp_dec.pdf")